# Final Project MACHINE LEARNING

---
* RM352122 Guilherme Ruy
* RM350785 Alexandra Maria Rodrigues Marques Figueira
* RM352152 Henrique da Silva Dergado
---

Dataset: https://www.kaggle.com/datasets/parisrohan/credit-score-classification

In [1]:
# Nesse passo, estamos importando as bibliotecas necessários para criar o modelo de classificação baseado no dataset de credit score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

In [2]:
#Fazer upload dos arquivos, selecionando o dataset de treino
#Baixar Dataset desse link(base TRAIN em uso apenas) https://www.kaggle.com/datasets/parisrohan/credit-score-classification?resource=download
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.csv to train.csv
User uploaded file "train.csv" with length 31136044 bytes


In [3]:
# Carregar o dataset e gravar ele dentro do dataframe.
df = pd.read_csv("train.csv" , delimiter=",", header=0)

<ipython-input-3-c6af8da02824>:2: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv" , delimiter=",", header=0)


In [4]:
# Inicializar o LabelEncoder
encoder = LabelEncoder()

In [5]:
#Verificando os dados do dataset
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Monthly_Inhand_Salary,84998.0,4194.170850,3183.686167,303.645417,1625.568229,3093.745000,5957.448333,15204.633333
Num_Bank_Accounts,100000.0,17.091280,117.404834,-1.000000,3.000000,6.000000,7.000000,1798.000000
Num_Credit_Card,100000.0,22.474430,129.057410,0.000000,4.000000,5.000000,7.000000,1499.000000
Interest_Rate,100000.0,72.466040,466.422621,1.000000,8.000000,13.000000,20.000000,5797.000000
Delay_from_due_date,100000.0,21.068780,14.860104,-5.000000,10.000000,18.000000,28.000000,67.000000
Num_Credit_Inquiries,98035.0,27.754251,193.177339,0.000000,3.000000,6.000000,9.000000,2597.000000
Credit_Utilization_Ratio,100000.0,32.285173,5.116875,20.000000,28.052567,32.305784,36.496663,50.000000
Total_EMI_per_month,100000.0,1403.118217,8306.041270,0.000000,30.306660,69.249473,161.224249,82331.000000


In [6]:
# Verificando amostragem de dados do dataset treino
print(df.head())

       ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utilization_Ratio     C

In [7]:
#Observando o tipo de cada coluna (Ainda precisa ser trabalhado para eliminar/converter as colunas FLOAT)
dtypes = pd.DataFrame(df.dtypes, columns = ['dtypes'])
dtypes

,dtypes
ID,object
Customer_ID,object
Month,object
Name,object
Age,object
SSN,object
Occupation,object
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64


In [8]:
#Observando na análise exploratória quantos registros estão em branco no dataset
df.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [9]:
# Dropando as colunas que não são relevantes ou que possuem muitos dados em branco, para o modelo de classificação de score
df = df.drop(columns= ["ID", "Customer_ID", "Month", "Name", "SSN", "Num_Credit_Card"])

In [10]:
#Preencher valores faltantes do campo com base nos valores já preenchidos
df['Monthly_Inhand_Salary'] = df.groupby('Occupation')['Monthly_Inhand_Salary'].transform(lambda x: x.ffill())

In [11]:
# Tratando nossos dados da coluna 'Occupation' em dummy
df['Occupation'] = df['Occupation'].str.replace('_______', 'Not Informed')
df = pd.get_dummies(df, columns=['Occupation'])

In [12]:
# Remover underscores e sinais de menos e preenchendo valores em brancos com a média
df['Age'] = pd.to_numeric(df['Age'].str.replace('_', '').replace('-', ''))
df['Changed_Credit_Limit'] = pd.to_numeric(df['Changed_Credit_Limit'].str.replace('_', '').replace('-', ''))

# Substituir valores acima de 100 pela média total
media_idade = df['Age'].mean()
df['Age'] = df['Age'].apply(lambda x: media_idade if x > 100 else x)

In [13]:
# Removendo linhas aonde 'Credit Mix' está invalidamente preenchido
df = df[df['Credit_Mix'] != '_']

# Ajustar o encoder aos dados e transformar a coluna numerica sequencial - CREDIT MIX
df['Credit_Mix_Num'] = encoder.fit_transform(df['Credit_Mix'])
df = df.drop('Credit_Mix', axis=1)

<ipython-input-13-04b7e27048b9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Credit_Mix_Num'] = encoder.fit_transform(df['Credit_Mix'])


In [14]:
# Removendo linhas aonde 'Payment_Behaviour' está invalidamente preenchido e transformando o restante em dummy
df = df[df['Payment_Behaviour'] != '!@9#%8']
df = pd.get_dummies(df, columns=['Payment_Behaviour'])

Coluna Credit_History_Age precisa ser quebrada em duas para manter somente anos e meses (numerico) e converter atraves de função para ter a mesma grandeza

In [15]:
# Ajustar o encoder aos dados e transformar a coluna numerica sequencial - CREDIT HISTORY AGE
df['Credit_History_Age_Num'] = encoder.fit_transform(df['Credit_History_Age'])
df = df.drop('Credit_History_Age', axis=1)

In [16]:
# Ajustar o encoder aos dados e transformar a coluna numerica sequencial - Paymente of Minimum Amount
df['Payment_of_Min_Amount_Num'] = encoder.fit_transform(df['Payment_of_Min_Amount'])
df = df.drop('Payment_of_Min_Amount', axis=1)

In [17]:
#Preencher valores em branco na coluna Type of Loan com valor 'Não especificado' e gerar valor númerico dos tipos para treino do modelo.
preencher_branco_TOL = 'Not Specified'
df['Type_of_Loan'] = df['Type_of_Loan'].fillna(preencher_branco_TOL)

# Ajustar o encoder aos dados e transformar a coluna numerica sequencial - TYPE OF LOAN
df['Type_Of_Loan_Num'] = encoder.fit_transform(df['Type_of_Loan'])
df = df.drop('Type_of_Loan', axis=1)

In [18]:
#Observando na análise exploratória quantos registros AINDA estão em branco no dataset
df.isnull().sum()

Age                                                      0
Annual_Income                                            0
Monthly_Inhand_Salary                                    2
Num_Bank_Accounts                                        0
Interest_Rate                                            0
Num_of_Loan                                              0
Delay_from_due_date                                      0
Num_of_Delayed_Payment                                5149
Changed_Credit_Limit                                  1558
Num_Credit_Inquiries                                  1443
Outstanding_Debt                                         0
Credit_Utilization_Ratio                                 0
Total_EMI_per_month                                      0
Amount_invested_monthly                               3336
Monthly_Balance                                        882
Credit_Score                                             0
Occupation_Accountant                                   

In [19]:
#Remover linhas que possuem valores em brancos
df = df.dropna()

In [20]:
#Remover qualquer underscore remanescente de colunas que não precisa substituir valores.
def remove_underscore(valor):
    if isinstance(valor, str):
        return valor.replace('_', '')
    else:
        return valor

df_sem_underscore = df.applymap(remove_underscore)

In [21]:
#Removendo valores e ajustando o formato das colunas
df_sem_underscore['Num_of_Loan'] = df_sem_underscore['Num_of_Loan'].str.extract('(\d+)').astype(float).astype(int)
df_sem_underscore['Num_of_Delayed_Payment'] = df_sem_underscore['Num_of_Delayed_Payment'].str.extract('(\d+)').astype(float).astype(int)
df_sem_underscore['Annual_Income'] = df_sem_underscore['Annual_Income'].astype(str)
df_sem_underscore['Annual_Income'] = df_sem_underscore['Annual_Income'].str.replace(r'[^0-9.]', '', regex=True)
df_sem_underscore['Outstanding_Debt'] = df_sem_underscore['Outstanding_Debt'].astype(str)
df_sem_underscore['Outstanding_Debt'] = df_sem_underscore['Outstanding_Debt'].str.replace(r'[^0-9.]', '', regex=True)

In [22]:
#Convertendo colunas para NUMERICO apenas usando o pandas
df_sem_underscore['Annual_Income'] = pd.to_numeric(df_sem_underscore['Annual_Income'], errors='coerce')
df_sem_underscore['Outstanding_Debt'] = pd.to_numeric(df_sem_underscore['Outstanding_Debt'], errors='coerce')
df_sem_underscore['Changed_Credit_Limit'] = pd.to_numeric(df_sem_underscore['Changed_Credit_Limit'], errors='coerce')
df_sem_underscore['Amount_invested_monthly'] = pd.to_numeric(df_sem_underscore['Amount_invested_monthly'], errors='coerce')
df_sem_underscore['Amount_invested_monthly'] = pd.to_numeric(df_sem_underscore['Amount_invested_monthly'], errors='coerce')
df_sem_underscore['Monthly_Balance'] = pd.to_numeric(df_sem_underscore['Monthly_Balance'], errors='coerce')

In [23]:
#Convertendo a coluna targer/alvo em númerico - sem drop
df_sem_underscore['Credit_Score'] = encoder.fit_transform(df['Credit_Score'])
df_sem_underscore['Credit_Score'] = pd.to_numeric(df_sem_underscore['Credit_Score'], errors='coerce')

In [24]:
import numpy as np

# Filtrar apenas as colunas numéricas
numeric_columns = df_sem_underscore.select_dtypes(include=[np.number]).columns

# Removendo outliers das colunas numéricas que podem atrapalhar a acurácia do modelo
percentile_threshold = 0.95
percentiles = df_sem_underscore[numeric_columns].quantile(percentile_threshold)

for column in numeric_columns:
    df_sem_underscore = df_sem_underscore[df_sem_underscore[column] <= percentiles[column]]

In [25]:
#Observando o tipo de cada coluna (Ainda precisa ser trabalhado para eliminar/converter as colunas FLOAT)
dtypes = pd.DataFrame(df.dtypes, columns = ['dtypes'])
dtypes

,dtypes
Age,float64
Annual_Income,object
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64
Interest_Rate,int64
Num_of_Loan,object
Delay_from_due_date,int64
Num_of_Delayed_Payment,object
Changed_Credit_Limit,float64
Num_Credit_Inquiries,float64


In [26]:
#Conferindo os valores para identificar outliers
df_sem_underscore.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,33495.0,2.818707e+01,5.091420e+01,-5.000000e+02,24.000000,33.000000,41.000000,53.000000
Annual_Income,33495.0,4.348958e+04,2.967296e+04,7.006035e+03,19180.870000,34534.820000,62901.080000,132606.690000
Monthly_Inhand_Salary,33495.0,3.630891e+03,2.482112e+03,3.036454e+02,1610.952500,2890.326667,5253.103333,10812.432500
Num_Bank_Accounts,33495.0,5.113450e+00,2.455436e+00,-1.000000e+00,3.000000,5.000000,7.000000,10.000000
Interest_Rate,33495.0,1.322615e+01,7.930623e+00,1.000000e+00,7.000000,12.000000,18.000000,33.000000
Num_of_Loan,33495.0,3.180355e+00,2.202309e+00,0.000000e+00,2.000000,3.000000,4.000000,9.000000
Delay_from_due_date,33495.0,1.815014e+01,1.173090e+01,-5.000000e+00,9.000000,16.000000,25.000000,54.000000
Num_of_Delayed_Payment,33495.0,1.265380e+01,5.754526e+00,0.000000e+00,9.000000,13.000000,17.000000,24.000000
Changed_Credit_Limit,33495.0,9.514470e+00,5.557068e+00,-6.440000e+00,5.270000,9.170000,13.330000,23.590000
Num_Credit_Inquiries,33495.0,5.218480e+00,3.454570e+00,0.000000e+00,2.000000,5.000000,8.000000,13.000000


In [27]:
#Observando o tipo de cada coluna (Ainda precisa ser trabalhado para eliminar/converter as colunas FLOAT)
dtypes = pd.DataFrame(df_sem_underscore.dtypes, columns = ['dtypes'])
dtypes

,dtypes
Age,float64
Annual_Income,float64
Monthly_Inhand_Salary,float64
Num_Bank_Accounts,int64
Interest_Rate,int64
Num_of_Loan,int64
Delay_from_due_date,int64
Num_of_Delayed_Payment,int64
Changed_Credit_Limit,float64
Num_Credit_Inquiries,float64


##Treinando modelo RandomForest

In [28]:
# Criando dois dataframes que serão usados para o treino do modelo,
# dropando a coluna target que nesse caso é o CREDIT SCORE
X = df_sem_underscore.drop("Credit_Score", axis=1)
y = df_sem_underscore["Credit_Score"]

In [29]:
# Dividindo os dataset para treino e teste do modelo (usando pareto 80/20 e semente aleatória fixa)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Criar um modelo de classificação Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [31]:
# Treinando o modelo
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [32]:
# Fazer a predição do modelo da base de teste
y_pred = model.predict(X_test)

In [33]:
# Avaliar o modelo
accuracia = accuracy_score(y_test, y_pred)
classificacao = classification_report(y_test, y_pred)

In [34]:
# Imprimir os valores da cassificação
print(f"Acurácia do modelo de score: {accuracia}")

print("Relatório de Classificação:")

print(classificacao)

Acurácia do modelo de score: 0.7822062994476787
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.71      0.72      0.71      1234
           1       0.76      0.76      0.76      1682
           2       0.82      0.81      0.82      3783

    accuracy                           0.78      6699
   macro avg       0.76      0.76      0.76      6699
weighted avg       0.78      0.78      0.78      6699



O algoritmo Random Forecast apresentou uma acurácia de 78%. Além disto, contém precisão e recall balanceados, sendo um algoritmo interessante para seguir com o aprendizado.

##Treinando modelo de Regressão Logística

In [35]:
# Criando dois dataframes que serão usados para o treino do modelo,
# dropando a coluna target que nesse caso é o CREDIT SCORE
X_rl = df_sem_underscore.drop("Credit_Score", axis=1)
y_rl = df_sem_underscore["Credit_Score"]

In [36]:
# Dividindo os dataset para treino e teste do modelo (usando pareto 80/20 e semente aleatória fixa)
X_train_rl, X_test_rl, y_train_rl, y_test_rl = train_test_split(X_rl, y_rl, test_size=0.2, random_state=42)

In [37]:
# Criar um modelo de classificação Regressão Logistica
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)

In [38]:
# Treinando o modelo
model.fit(X_train_rl, y_train_rl)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [39]:
# Fazer a predição do modelo da base de teste
y_pred_rl = model.predict(X_test_rl)

In [40]:
# Avaliar o modelo

accuracia_rl = accuracy_score(y_test_rl, y_pred_rl)
classificacao_rl = classification_report(y_test_rl, y_pred_rl, zero_division=1)
precision_rl = precision_score(y_test_rl, y_pred_rl, average=None, zero_division=1)
f1_rl = f1_score(y_test_rl, y_pred_rl, average=None, zero_division=1)

In [41]:
# Imprimir os valores da cassificação
print(f"Acurácia do modelo de score: {accuracia_rl}")

print("Relatório de Classificação:")

print(classificacao_rl)

Acurácia do modelo de score: 0.251530079116286
Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00      1234
           1       0.25      1.00      0.40      1682
           2       1.00      0.00      0.00      3783

    accuracy                           0.25      6699
   macro avg       0.75      0.33      0.13      6699
weighted avg       0.81      0.25      0.10      6699



A Regressão Logística apresentou uma acurácia baixa de 25% e, além disto, precisão e recall estão desbalanceados. Este algoritmo não está com configuração e qualidade de aprendizado adequadas.

## Testando Ensamble

In [42]:
# Criando dois dataframes que serão usados para o treino do modelo,
# dropando a coluna target que nesse caso é o CREDIT SCORE
X_ens = df_sem_underscore.drop("Credit_Score", axis=1)
y_ens = df_sem_underscore["Credit_Score"]

In [43]:
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier


# Instanciando os modelos
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
mlp_clf = MLPClassifier(random_state=42)
svc_clf = SVC(probability=True)  # habilitar probabilidade para soft voting

In [ ]:
# Dividindo os dataset para treino e teste do modelo (usando pareto 80/20 e semente aleatória fixa)
X_train_ens, X_test_ens, y_train_ens, y_test_ens = train_test_split(X_ens, y_ens, test_size=0.2, random_state=42)

In [ ]:
# Criar um ensemble de votação com os modelos individuais
ensemble_model = VotingClassifier(estimators=[
    ('gb', gb_clf),
    ('svc', svc_clf),
    ('mlp', mlp_clf)
], voting='soft')  # Testamos com 'hard' e 'soft' e o resultado deu quase o mesmo

# Treinar o ensemble
ensemble_model.fit(X_train_ens, y_train_ens)

VotingClassifier(estimators=[('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('svc', SVC(probability=True)),
                             ('mlp', MLPClassifier(random_state=42))],
                 voting='soft')

In [ ]:
# Fazer previsões com o ensemble
ensemble_pred = ensemble_model.predict(X_test_ens)

print (classification_report(y_test_ens, ensemble_pred))

              precision    recall  f1-score   support

           0       0.52      0.41      0.46      1234
           1       0.69      0.46      0.56      1682
           2       0.67      0.82      0.74      3783

    accuracy                           0.66      6699
   macro avg       0.63      0.56      0.58      6699
weighted avg       0.65      0.66      0.64      6699



##Feedback

Após a análise exploratória dos dados, iniciamos a limpeza e o preparo adequado da base de dados para garantir a qualidade e a confiabilidade dos modelos de classificação implementados.

Diversos algoritmos foram testados, incluindo RandomForest, Regressão Logística e um ensemble soft e hard voting com GradientBoostingClassifier, MLPClassifier e SVC, com o objetivo de identificar o método mais eficaz para a classificação de credit score.

Os modelos foram avaliados com base em sua acurácia, precisão, recall e f1-score, permitindo uma análise comparativa entre os diferentes classificadores.

O **modelo RandomForestClassifier destacou-se** como o mais eficiente, apresentando não apenas a melhor performance em termos de métricas de avaliação, mas também uma robustez significativa frente às variações nos dados de teste.

A escolha do RandomForestClassifier como o melhor modelo para esta aplicação específica deve-se à sua natureza versátil, que combina múltiplas árvores de decisão para produzir uma estimativa mais estável e precisa.